# References
* https://dadosabertos.camara.leg.br/swagger/api.html
* https://www.youtube.com/watch?v=t4C1c62Z4Ag
* https://docs.sqlalchemy.org/en/20/dialects/mysql.html#module-sqlalchemy.dialects.mysql.base
* https://docs.sqlalchemy.org/en/20/changelog/whatsnew_20.html#migrating-an-existing-mapping

In [1]:
import httpx
import pandas as pd
from tqdm import tqdm

# 1. Introdução
Neste notebook, iremos explorar a API de dados abertos da Câmara dos Deputados do Brasil. A API fornece informações sobre deputados, partidos, proposições, votações, etc.

Serão utilizadas as bibliotecas httpx, pandas e sqlalchemy para acessar, manipular e armazenar os dados.

Primeiramente, por meio da documentação da API, vamos entender como acessar os dados de deputados.

# 2. Conhecendo a API

In [2]:
base_url = "https://dadosabertos.camara.leg.br/api/v2"

Usando o httpx, vamos fazer uma requisição GET para o endpoint /deputados para exibir a lista de deputados.

In [3]:
httpx.get(f"{base_url}/deputados").json()

{'dados': [{'id': 220593,
   'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/220593',
   'nome': 'Abilio Brunini',
   'siglaPartido': 'PL',
   'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/37906',
   'siglaUf': 'MT',
   'idLegislatura': 57,
   'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/220593.jpg',
   'email': 'dep.abiliobrunini@camara.leg.br'},
  {'id': 204379,
   'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/204379',
   'nome': 'Acácio Favacho',
   'siglaPartido': 'MDB',
   'uriPartido': 'https://dadosabertos.camara.leg.br/api/v2/partidos/36899',
   'siglaUf': 'AP',
   'idLegislatura': 57,
   'urlFoto': 'https://www.camara.leg.br/internet/deputado/bandep/204379.jpg',
   'email': 'dep.acaciofavacho@camara.leg.br'},
  {'id': 220714,
   'uri': 'https://dadosabertos.camara.leg.br/api/v2/deputados/220714',
   'nome': 'Adail Filho',
   'siglaPartido': 'REPUBLICANOS',
   'uriPartido': 'https://dadosabertos.camara.leg.br/api/v

Agora que sabemos como podemos fazer requisições para a API, irei criar duas funções que fazem requisições para 2 endpoints que usaremos nesse trabalho:
* /deputados
* /deputados/{id}/despesas?ano={year}

In [4]:
def get_deputados() -> dict:
    return httpx.get(f"{base_url}/deputados").json()


def get_despesas(id: int, year: int = 2022) -> dict:
    response = httpx.get(f"{base_url}/deputados/{id}/despesas?ano={year}")
    return response.json()

De acordo com as especificações do trabalho, precisamos consultar a base de gastos da Câmara no ano de 2022 e armazenar tudo isso em um banco de dados relacional.
  
Primeiramente, irei consultar o que é retornado pela API no endpoint das despesas de um deputado.

In [5]:
get_despesas(id=136811, year=2022)

{'dados': [{'ano': 2022,
   'mes': 12,
   'tipoDespesa': 'LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES',
   'codDocumento': 7499778,
   'tipoDocumento': 'Nota Fiscal',
   'codTipoDocumento': 0,
   'dataDocumento': '2023-02-17T00:00:00',
   'numDocumento': '635',
   'valorDocumento': 3090.0,
   'urlDocumento': 'https://www.camara.leg.br/cota-parlamentar/documentos/publ/1758/2023/7499778.pdf',
   'nomeFornecedor': 'LIZIANE QUEVEDO',
   'cnpjCpfFornecedor': '07086926000190',
   'valorLiquido': 3090.0,
   'valorGlosa': 0.0,
   'numRessarcimento': '',
   'codLote': 1909470,
   'parcela': 0},
  {'ano': 2022,
   'mes': 12,
   'tipoDespesa': 'LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES',
   'codDocumento': 7499776,
   'tipoDocumento': 'Nota Fiscal',
   'codTipoDocumento': 0,
   'dataDocumento': '2023-02-17T00:00:00',
   'numDocumento': '636',
   'valorDocumento': 5610.0,
   'urlDocumento': 'https://www.camara.leg.br/cota-parlamentar/documentos/publ/1758/2023/7499776.pdf',
   'nomeFornecedor

Podemos ver que a API retorna um JSON, e dentro da chave 'dados' temos uma lista de despesas. Cada despesa é um dicionário com várias informações, que são:
* ano
* cnpjCpfFornecedor"
* codDocumento
* codLote
* codTipoDocumento
* dataDocumento
* mes
* nomeFornecedor
* numDocumento
* numRessarcimento
* parcela
* tipoDespesa
* tipoDocumento
* urlDocumento
* valorDocumento
* valorGlosa
* valorLiquido
  
Logo, podemos criar uma tabela no banco de dados para armazenar essas informações.

# 3. Criando o banco de dados

Aqui usaremos o Postgresql para criar o banco de dados e a tabela que armazenará as despesas dos deputados e o id do deputado.  

Aqui criaremos a tabela que armazenará as despesas dos deputados. Essa tabela será uma classe que herda de Base, que é uma classe base do SQLAlchemy.

In [6]:
from sqlalchemy import Column, Float, Integer, String, create_engine
from sqlalchemy.orm import DeclarativeBase, sessionmaker

# Create a SQLite database engine
user = "lucas"
password = "12345"
host = "localhost:5432"
database = "camara"

engine = create_engine(f"postgresql://{user}:{password}@{host}/{database}")


# Create a base class for our declarative class definitions
class Base(DeclarativeBase):
    pass


# Define a simple table model
class Despesas(Base):
    __tablename__ = "despesas"
    id = Column(Integer, primary_key=True)
    id_deputado = Column(Integer)
    ano = Column(Integer)
    cnpj_cpf_fornecedor = Column(String)
    cod_documento = Column(String)
    cod_lote = Column(String)
    cod_tipo_documento = Column(String)
    data_documento = Column(String)
    mes = Column(Integer)
    nome_fornecedor = Column(String)
    num_documento = Column(String)
    num_ressarcimento = Column(String)
    parcela = Column(String)
    tipo_despesa = Column(String)
    tipo_documento = Column(String)
    url_documento = Column(String)
    valor_documento = Column(Float)
    valor_glosa = Column(Float)
    valor_liquido = Column(Float)


class Deputados(Base):
    __tablename__ = "deputados"
    id = Column(Integer, primary_key=True)
    nome = Column(String)
    sigla_partido = Column(String)
    sigla_uf = Column(String)
    url_foto = Column(String)
    email = Column(String)


# Drop tables if they exist
# Base.metadata.drop_all(engine)

# Create the tables in the database
Base.metadata.create_all(engine)

# Create a session to interact with the database
Session = sessionmaker(bind=engine)
session = Session()

Para adicionar os dados no banco de dados, primeiro criamos um novo objeto da classe Despesas e passamos os valores que queremos adicionar.

In [7]:
despesa = Despesas(
    id_deputado=136811,
    ano=2022,
    mes=12,
    tipo_despesa="LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES",
    cod_documento=7499778,
    tipo_documento="Nota Fiscal",
    cod_tipo_documento=0,
    data_documento="2023-02-17T00:00:00",
    num_documento="635",
    valor_documento=3090.0,
    url_documento="https://www.camara.leg.br/cota-parlamentar/documentos/publ/1758/2023/7499778.pdf",
    nome_fornecedor="LIZIANE QUEVEDO",
    cnpj_cpf_fornecedor="07086926000190",
    valor_liquido=3090.0,
    valor_glosa=0.0,
    num_ressarcimento="",
    cod_lote=1909470,
    parcela=0,
)

Logo após isso, criamos uma sessão e adicionamos o objeto criado. Por fim, fazemos um commit para salvar as alterações no banco de dados. (comentarei tudo isso no código)

In [8]:
# session = Session()
# session.add(despesa)
# session.commit()
# session.close()

Outro ponto importante, é que se temos um dicionário com as informações das despesas, podemos passar esse dicionário diretamente para a função que cria o objeto da classe Despesas. Isso é feito com o operador **, que desempacota o dicionário e passa cada chave como argumento para a função.

In [9]:
dados = {
    "ano": 2022,
    "mes": 12,
    "tipoDespesa": "LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES",
    "codDocumento": 7499778,
    "tipoDocumento": "Nota Fiscal",
    "codTipoDocumento": 0,
    "dataDocumento": "2023-02-17T00:00:00",
    "numDocumento": "635",
    "valorDocumento": 3090.0,
    "urlDocumento": "https://www.camara.leg.br/cota-parlamentar/documentos/publ/1758/2023/7499778.pdf",
    "nomeFornecedor": "LIZIANE QUEVEDO",
    "cnpjCpfFornecedor": "07086926000190",
    "valorLiquido": 3090.0,
    "valorGlosa": 0.0,
    "numRessarcimento": "",
    "codLote": 1909470,
    "parcela": 0,
}

despesa = Despesas(id_deputado=136811, **dados)
despesa

Com isso em mente, podemos popular o nosso banco de dados com as despesas de todos os deputados. 

In [10]:
deputados = get_deputados()
ids = [deputado["id"] for deputado in deputados["dados"]]

In [11]:
for deputado in tqdm(deputados.get("dados")):
    try:
        temp_deputado = Deputados(
            id=deputado.get("id"),
            nome=deputado.get("nome"),
            siglaPartido=deputado.get("siglaPartido"),
            siglaUf=deputado.get("siglaUf"),
            urlFoto=deputado.get("urlFoto"),
            email=deputado.get("email"),
        )
        session.add(temp_deputado)
    except Exception as e:
        print(f"Erro ao inserir deputado: {despesa}")
        print(e)

session.commit()
session.close()

100%|██████████| 513/513 [00:00<00:00, 29314.81it/s]


In [12]:
for id in tqdm(ids):
    despesas = get_despesas(id=id, year=2022)
    for despesa in despesas["dados"]:
        try:
            temp_despesa = Despesas(id_deputado=id, **despesa)
            session.add(temp_despesa)
        except Exception as e:
            print(f"Erro ao inserir despesa: {despesa}")
            print(e)

session.commit()
session.close()

100%|██████████| 513/513 [02:10<00:00,  3.94it/s]       


In [13]:
# Create a session
session = Session()

# Query all users from the database
rows = session.query(Despesas).count()

session.close()

print(f"\nTotal de despesas: {rows}")


Total de despesas: 33


In [14]:
# Create a session
session = Session()

# Query all users from the database
rows = session.query(Deputados).count()

session.close()

print(f"\nTotal de deputados: {rows}")


Total de deputados: 513


Para se ter uma rápida visualização dos dados, podemos gerar um dataframe do pandas com todos os registros da tabela de despesas.

In [15]:
# all registers as pandas dataframe
df = pd.read_sql("despesas", engine)
df.head()

,id,id_deputado,ano,cnpjCpfFornecedor,codDocumento,codLote,codTipoDocumento,dataDocumento,mes,nomeFornecedor,numDocumento,numRessarcimento,parcela,tipoDespesa,tipoDocumento,urlDocumento,valorDocumento,valorGlosa,valorLiquido
0,1,136811,2022,07086926000190,7499778,1909470,0,2023-02-17T00:00:00,12,LIZIANE QUEVEDO,635,,0,LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,Nota Fiscal,https://www.camara.leg.br/cota-parlamentar/doc...,3090.0,0.0,3090.0
1,2,136811,2022,07086926000190,7499776,1909470,0,2023-02-17T00:00:00,12,LIZIANE QUEVEDO,636,,0,LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES,Nota Fiscal,https://www.camara.leg.br/cota-parlamentar/doc...,5610.0,0.0,5610.0
2,3,178835,2022,11405562000131,7499669,1909407,4,2023-02-15T00:00:00,12,NOVA IMPRESSAO GRAFICA E PAPELARIA LTDA,1690,,0,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,Nota Fiscal Eletrônica,http://www.camara.leg.br/cota-parlamentar/nota...,2090.0,0.0,2090.0
3,4,109429,2022,37894749000130,7497764,1908359,0,2023-02-09T00:00:00,12,ELDORADO COMUNICACAO E JORNALISMO LTDA,21,,0,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,Nota Fiscal,https://www.camara.leg.br/cota-parlamentar/doc...,1500.0,0.0,1500.0
4,5,204507,2022,42263664000193,7497940,1908797,0,2023-02-10T00:00:00,12,ALINE DA SILVA SANTOS NASCIMENTO 35683981840,24,,0,DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.,Nota Fiscal,https://www.camara.leg.br/cota-parlamentar/doc...,2000.0,0.0,2000.0
